In [1]:
import pandas as pd

# Load recipe data from CSV
recipes_df = pd.read_csv('../Resource/completed_recipes.csv')
print(f"Loaded {len(recipes_df)} recipes.")


Loaded 522517 recipes.


In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "D*d4-0+Kl+lxfbbzh5ut"),
    ca_certs="~/http_ca.crt"
)

if es.ping():
    print("Elasticsearch connect for now!!")
else:
    print("failed to connect")

Elasticsearch connect for now!!


In [3]:
import json
import numpy as np
from elasticsearch.helpers import bulk
import re

# Define index name and sample size for development
index_name = "recipes"
sample_size = 1000  # Set the sample size for testing (adjust as needed)

# Delete the index if it already exists
es.indices.delete(index=index_name, ignore=[400, 404])

mapping = {
    "settings": {
        "analysis": {
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "ngram",
                    "min_gram": 2,  # Minimum length of n-grams
                    "max_gram": 3,  # Maximum length of n-grams
                    "token_chars": ["letter", "digit"]
                }
            },
            "analyzer": {
                "default": {
                    "type": "english"
                },
                "ngram_analyzer": {  # Custom N-gram analyzer for partial word search
                    "type": "custom",
                    "tokenizer": "ngram_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "recipe_id": {"type": "keyword"},
            "name": { 
                "type": "text", 
                "analyzer": "english",
                "fields": { 
                    "ngram": {  # N-gram variant for better partial matches
                        "type": "text", 
                        "analyzer": "ngram_analyzer"
                    }
                }
            },
            "description": {"type": "text", "analyzer": "english"},
            "instructions": {"type": "text", "analyzer": "english"},
            "text": {"type": "text", "analyzer": "english"},
            "calories": {"type": "float"},
            "rating": {"type": "float"},
            "image_url": {"type": "keyword"}
        }
    }
}

es.indices.create(index=index_name, body=mapping)
print(f"Created index: {index_name}")

recipes_sample = recipes_df.head(sample_size)

def clean_text(text):
    """Remove unwanted characters like c("..."), quotes, and escape sequences."""
    if not isinstance(text, str):  
        return ""  

    text = re.sub(r'c\("', '', text)
    text = re.sub(r'"\)', '', text)

    text = text.replace('\\"', '')  
    text = text.replace('"', '')    
    text = text.replace("\\", '')  
    cleaned_urls = re.sub(r'\s+', ' ', text.strip())
    # Split the string by ', ' (comma followed by a space)
    urls = cleaned_urls.split(', ')
    return urls

def clean_instructions_combined_v2(instructions):
    """Cleans and formats recipe instructions into a single, well-structured sentence."""
    if isinstance(instructions, list):
        instructions = " ".join(instructions)  # Join list into a single string
    
    if not isinstance(instructions, str):
        return ""

    # Remove unwanted wrapping (c(...)) and extra quotes
    instructions = re.sub(r'c\s*\(\s*', '', instructions)  # Remove "c("
    instructions = re.sub(r'\s*\)$', '', instructions)  # Remove trailing ")"
    instructions = instructions.strip('"')

    # Fix letter-by-letter spacing issues (e.g., "M i x" → "Mix in")
    words = instructions.split()  # Split into words
    cleaned_words = []
    buffer = ""

    for word in words:
        if len(word) == 1:  # If it's a single letter, buffer it
            buffer += word
        else:
            if buffer:
                cleaned_words.append(buffer + word)  # Merge buffer with current word
                buffer = ""
            else:
                cleaned_words.append(word)

    if buffer:  # Append remaining buffered text
        cleaned_words.append(buffer)

    instructions = " ".join(cleaned_words)

    # Remove redundant punctuation and ensure proper spacing
    instructions = re.sub(r'\.\s*\.', '.', instructions)  # Remove repeated periods
    instructions = re.sub(r'\s*\.\s*', '. ', instructions)  # Ensure proper space after periods
    instructions = re.sub(r'\s*,', ',', instructions)  # Remove spaces before commas

    # Remove leading/trailing unwanted characters (like extra quotes)
    instructions = re.sub(r'(^\"|\"$)', '', instructions)  # Remove leading/trailing quotes

    # Ensure proper formatting of sentences and remove unnecessary escape sequences
    instructions = instructions.replace('\\"', '')  # Remove escaped quotes
    instructions = instructions.replace('", "', ', ')  # Convert improperly formatted list items into a natural sentence

    # Convert sentence breaks into commas for a continuous explanation
    instructions = re.sub(r'\s*\.\s*', ', ', instructions)  # Convert periods into commas for a smoother flow

    # Fix double commas and extra spaces
    instructions = re.sub(r',\s*,+', ', ', instructions)  # Remove repeated commas
    instructions = re.sub(r'\s+', ' ', instructions)  # Remove extra spaces

    # Ensure only one period at the very end
    instructions = instructions.strip().rstrip(',') + "."  # Remove trailing comma and add a period

    return instructions



def generate_docs(df):
    for idx, row in df.iterrows():
        recipe_id = str(int(float(row.get('RecipeId', idx))))
        name = row.get('Name', '')
        description = row.get('Description', '')  # Replacing "ingredients"
        instructions = clean_instructions_combined_v2(row.get('RecipeInstructions', []))  # ✅ Clean instructions at indexing
        text = clean_text(row.get('text', ''))  # Clean text
        calories = float(row.get('Calories', 0))
        image_url = row.get('image_link', [])

        try:
            rating = float(row['AggregatedRating']) if not np.isnan(row['AggregatedRating']) else 0
        except (KeyError, TypeError, ValueError):
            rating = 0
        
        doc = {
            "_op_type": "index",
            "_index": index_name,
            "_id": recipe_id,
            "_source": {
                "recipe_id": recipe_id,
                "name": name,
                "description": description,  # Updated field
                "instructions": instructions,  # ✅ Cleaned instructions
                "text": text,  # Updated field
                "calories": calories,
                "rating": rating,
                "image_url": image_url
            }
        }
        if not np.isnan(row.get('AggregatedRating', np.nan)):
            doc["_source"]["rating"] = float(row["AggregatedRating"])
        yield doc


bulk(es, generate_docs(recipes_sample))

print(f"Indexed {len(recipes_sample)} recipes into Elasticsearch.")


C:\Users\Ned\AppData\Local\Temp\ipykernel_24548\3960969787.py:11: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])


Created index: recipes
Indexed 1000 recipes into Elasticsearch.


### Flask

In [4]:
from flask import Flask, request, jsonify
from elasticsearch import Elasticsearch
import re
from flask_cors import CORS

In [5]:
app = Flask(__name__)
CORS(app)

# Connect to Elasticsearch 
INDEX_NAME = "recipes"

@app.route('/search', methods=['GET'])
def search():
    """Search endpoint for querying recipes."""
    query = request.args.get('q', '')  
    if not query:
        return jsonify({"error": "Query parameter 'q' is required"}), 400
    
    # Elasticsearch search query
    es_query = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["name^3", "description^2", "text", "name.ngram^2"],  
                "fuzziness": "AUTO" 
            }
        }
    }
    
    response = es.search(index=INDEX_NAME, body=es_query)
    results = [
        {
            "recipe_id": hit["_source"].get("recipe_id"),
            "name": hit["_source"].get("name"),
            "description": hit["_source"].get("description"),  
            "text": hit["_source"].get("text"), 
            "image_url": clean_text(hit["_source"].get("image_url")),
            "calories": hit["_source"].get("calories"),
            "rating": hit["_source"].get("rating", 0),
            "score": hit["_score"]
            
        }
        for hit in response.get("hits", {}).get("hits", [])
    ]
    
    return jsonify({"results": results})

@app.route('/recipe/<recipe_id>', methods=['GET'])
def get_recipe(recipe_id):
    """Fetches a single recipe by ID."""
    response = es.get(index=INDEX_NAME, id=recipe_id, ignore=[404])
    
    if response and response.get("found"):
        source = response["_source"]
        
        # Clean instructions dynamically
        source["instructions"] = clean_instructions_combined_v2(source.get("instructions", ""))
        
        # Clean image URL using clean_text
        image_urls = clean_text(source.get("image_url", ""))
        source["image_url"] = image_urls[0] if image_urls else ""

        return jsonify(source)
    
    return jsonify({"error": "Recipe not found"}), 404



In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
